# August Karlstedt

In [39]:
%matplotlib inline

import os
import imp
import operator
import math
import glob
import json
import time

from IPython.display import IFrame

import nltk

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

#import pickle
#from six.moves import urllib

import tensorflow

from keras.models import Sequential
from keras.layers import Input, Dense, Activation, Dropout
from keras.layers import LSTM, TimeDistributed
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from keras import metrics
from keras.utils.np_utils import to_categorical

#import fasttext
# https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md

import gensim
# https://nlp.stanford.edu/projects/glove/

In [40]:
files = []
data = []
header = ['Annotation ID', 'Batch ID', 'Annotator ID', 'Policy ID', 'Segment ID', 'Category Name', 'Attributes/Values', 'Policy URL', 'Date']
keep_columns = ['Segment ID', 'Category Name', 'Attributes/Values']
for file in glob.glob("data\\annotations/*.csv"):
    files.append(file[17:-4])
    data.append(pd.read_csv(file, names=header)[keep_columns])

In [41]:
files[:10]

['1017_sci-news.com',
 '1028_redorbit.com',
 '1034_aol.com',
 '1050_honda.com',
 '105_amazon.com',
 '1070_wnep.com',
 '1083_highgearmedia.com',
 '1089_freep.com',
 '1099_enthusiastnetwork.com',
 '1106_allstate.com']

In [42]:
data[0]

,Segment ID,Category Name,Attributes/Values
0,0,Other,"{""Other Type"": {""selectedText"": ""Sci-News.com ..."
1,1,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""nformati..."
2,1,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""nformati..."
3,2,Data Retention,"{""Personal Information Type"": {""selectedText"":..."
4,3,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""Not sele..."
5,3,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""Not sele..."
6,3,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""Not sele..."
7,3,First Party Collection/Use,"{""Collection Mode"": {""selectedText"": ""Not sele..."
8,4,International and Specific Audiences,"{""Audience Type"": {""selectedText"": ""Sci-News.c..."
9,4,Other,"{""Other Type"": {""selectedText"": ""Parents or gu..."


In [43]:
policies = []
for file in files:
    with open("data\\sanitized_policies/{}.html".format(file)) as f:
        policies.append(f.readlines()[0].split('|||'))

In [44]:
print(len(policies), len(policies[0]), policies[0])

115 13 ['Privacy Policy <br> <br> Sci-News.com is committed to protecting and respecting your privacy. To better inform you of our policy concerning user privacy, we have adopted the following terms. Please note that these terms are subject to change, and any such changes will be included on this page. <br> <br>', 'Information that Sci-News.com May Collect Online <br> <br> Sci-News.com may collect and process the following data about you: <br> <br> - information that you provide by filling in forms on our site, including names, e-mail and website addresses; we may also ask you for information for other purposes, for example when you report a problem with our site; <br> <br>', '- if you contact us, we may keep a record of that correspondence; <br> <br>', '- details of your visits to our site including, but not limited to, traffic data, location data, weblogs and other communication data. <br> <br>', 'Sci-News.com does not knowingly collect or solicit personal information from anyone und

Okay, so we have loaded our CSV files and also our privacy policies. Now we can use the CSV data to index into each segment in each privacy policy. Let's try this out manually for one privacy policy just to start.

In [45]:
test_file_index = 64
test_csv_file = data[test_file_index] # our table of attribute/values/categories/segment ids
test_pp_file = policies[test_file_index]
files[test_file_index]

'414_washingtonian.com'

Okay, let's load a single annotation and display the highlighted text in the privacy policy

In [46]:
test_row_num = 4

In [47]:
test_segment_id = test_csv_file['Segment ID'][test_row_num]
test_segment_id

2

In [48]:
test_category_name = test_csv_file['Category Name'][test_row_num]
test_category_name

'First Party Collection/Use'

In [49]:
test_attributes_values = json.loads(test_csv_file['Attributes/Values'][test_row_num])
test_attributes_values

{'Action First-Party': {'endIndexInSegment': 44,
  'selectedText': 'we collect',
  'startIndexInSegment': 34,
  'value': 'Unspecified'},
 'Choice Scope': {'endIndexInSegment': -1,
  'selectedText': 'Not selected',
  'startIndexInSegment': -1,
  'value': 'Unspecified'},
 'Choice Type': {'endIndexInSegment': -1,
  'selectedText': 'Not selected',
  'startIndexInSegment': -1,
  'value': 'Unspecified'},
 'Collection Mode': {'endIndexInSegment': -1,
  'selectedText': 'null',
  'startIndexInSegment': -1,
  'value': 'not-selected'},
 'Does/Does Not': {'endIndexInSegment': -1,
  'selectedText': 'null',
  'startIndexInSegment': -1,
  'value': 'Does'},
 'Identifiability': {'endIndexInSegment': -1,
  'selectedText': 'null',
  'startIndexInSegment': -1,
  'value': 'not-selected'},
 'Personal Information Type': {'endIndexInSegment': 56,
  'selectedText': 'include',
  'startIndexInSegment': 49,
  'value': 'Other'},
 'Purpose': {'endIndexInSegment': -1,
  'selectedText': 'Not selected',
  'startIndexI

In [50]:
test_attributes_values.keys()

dict_keys(['User Type', 'Does/Does Not', 'Personal Information Type', 'Action First-Party', 'Collection Mode', 'Choice Scope', 'Purpose', 'Identifiability', 'Choice Type'])

In [51]:
test_start_index = test_attributes_values['Personal Information Type']['startIndexInSegment']
test_end_index = test_attributes_values['Personal Information Type']['endIndexInSegment']
test_value = test_attributes_values['Personal Information Type']['value']
test_selected_text = test_attributes_values['Personal Information Type']['selectedText']
print(test_start_index, test_end_index, test_value, test_selected_text)

49 56 Other include


In [52]:
test_pp_file[test_segment_id][test_start_index:test_end_index]

'include'

In [53]:
print("Segment ID: {}".format(test_segment_id))
print("Category Name: {}".format(test_category_name))
print("Type: {}".format('Personal Information Type'))
print("Value: {}".format(test_value))
print("Selected Text: {}".format(test_selected_text))

Segment ID: 2
Category Name: First Party Collection/Use
Type: Personal Information Type
Value: Other
Selected Text: include


Okay, a few things.

1. The text is already included in the CSV file. We don't necessarily need to index into the policy itself, unless we want to grab the entire sentence. Let's start by training the NN on *just* the selected text and not the whole sentence and see what we get.
2. The JSON in the `Attributes/Values` column has many different types. We need to grab them all so we know what to index into the dictionary for.
3. There's a value associated with each annotation OR it can be `Unspecified`

In [54]:
# categories = set()
# for datum in data:
#     cat = datum['Category Name']
#     categories.update(cat)
# categories

categories = [
 'Data Retention',
 'Data Security',
 'Do Not Track',
 'First Party Collection/Use', # 3
 'International and Specific Audiences',
 'Other',
 'Policy Change',
 'Third Party Sharing/Collection', # 7
 'User Access, Edit and Deletion',
 'User Choice/Control'
]

one_hot_categories = np.array([
#                       [0,0,0,0,0,0,0,0,0,1],
#                       [0,0,0,0,0,0,0,0,1,0],
#                       [0,0,0,0,0,0,0,1,0,0],
                      [0,1],#[0,0,0,0,0,0,1,0,0,0],
#                       [0,0,0,0,0,1,0,0,0,0],
#                       [0,0,0,0,1,0,0,0,0,0],
#                       [0,0,0,1,0,0,0,0,0,0],
                      [1,0]#,[0,0,1,0,0,0,0,0,0,0],
#                       [0,1,0,0,0,0,0,0,0,0],
#                       [1,0,0,0,0,0,0,0,0,0]
                    ])

cat_dict = {
#  categories[0]: one_hot_categories[0],
#  categories[1]: one_hot_categories[1],
#  categories[2]: one_hot_categories[2],
 categories[3]: one_hot_categories[0],##one_hot_categories[3],
#  categories[4]: one_hot_categories[4],
#  categories[5]: one_hot_categories[5],
#  categories[6]: one_hot_categories[6],
 categories[7]: one_hot_categories[1]#one_hot_categories[7],
#  categories[8]: one_hot_categories[8],
#  categories[9]: one_hot_categories[9]
}

Let's first get all of the Attributes/Values types to index into our dictionary. In the OPP-115 paper, Figure 1 looks like it shows the types on the right side.

In [55]:
# attribute_value_types = set()
# attribute_value_values = set()
# for datum in data:
#     avs = datum['Attributes/Values']
#     for row in avs:
#         parsed = json.loads(row)
#         keys = list(parsed.keys())
#         attribute_value_types.update(keys)
#         for key in keys:
#             attribute_value_values.add(parsed[key]['value'])

attribute_value_types = ['Access Scope',
 'Access Type',
 'Action First-Party',
 'Action Third Party',
 'Audience Type',
 'Change Type',
 'Choice Scope',
 'Choice Type',
 'Collection Mode',
 'Do Not Track policy',
 'Does/Does Not',
 'Identifiability',
 'Notification Type',
 'Other Type',
 'Personal Information Type',
 'Purpose',
 'Retention Period',
 'Retention Purpose',
 'Security Measure',
 'Third Party Entity',
 'User Choice',
 'User Type']

attribute_value_values = ['Additional service/feature',
 'Advertising',
 'Aggregated or anonymized',
 'Analytics/Research',
 'Basic service/feature',
 'Both',
 'Browser/device privacy controls',
 'Californians',
 'Children',
 'Citizens from other countries',
 'Collect from user on other websites',
 'Collect in mobile app',
 'Collect on first party website/app',
 'Collect on mobile website',
 'Collect on website',
 'Collection',
 'Computer information',
 'Contact',
 'Cookies and tracking elements',
 'Data access limitation',
 'Deactivate account',
 'Delete account (full)',
 'Delete account (partial)',
 'Demographic',
 'Does',
 'Does Not',
 'Dont use service/feature',
 'Edit information',
 'Europeans',
 'Explicit',
 'Export',
 'Financial',
 'First party collection',
 'First party use',
 'First-party privacy controls',
 'General notice in privacy policy',
 'General notice on website',
 'Generic',
 'Generic personal information',
 'Health',
 'Honored',
 'IP address and device IDs',
 'Identifiable',
 'Implicit',
 'In case of merger or acquisition',
 'Indefinitely',
 'Introductory/Generic',
 'Legal requirement',
 'Limited',
 'Location',
 'Marketing',
 'Mentioned, but unclear if honored',
 'Merger/Acquisition',
 'Named third party',
 'No notification',
 'Non-privacy relevant change',
 'None',
 'Not honored',
 'Not mentioned',
 'Opt-in',
 'Opt-out',
 'Opt-out link',
 'Opt-out via contacting company',
 'Other',
 'Other data about user',
 'Other part of company/affiliate',
 'Other users',
 'Perform service',
 'Personal identifier',
 'Personal notice',
 'Personalization/Customization',
 'Practice not covered',
 'Privacy contact information',
 'Privacy relevant change',
 'Privacy review/audit',
 'Privacy training',
 'Privacy/Security program',
 'Profile data',
 'Public',
 'Receive from other parts of company/affiliates',
 'Receive from other service/third-party (named)',
 'Receive from other service/third-party (unnamed)',
 'Receive/Shared with',
 'Secure data storage',
 'Secure data transfer',
 'Secure user authentication',
 'See',
 'Service Operation and Security',
 'Service operation and security',
 'Social media data',
 'Stated Period',
 'Survey data',
 'Third party sharing/collection',
 'Third party use',
 'Third-party privacy controls',
 'Track on first party website/app',
 'Track user on other websites',
 'Transactional data',
 'Unnamed third party',
 'Unspecified',
 'Use',
 'User Profile',
 'User account data',
 'User online activities',
 'User participation',
 'User profile',
 'User with account',
 'User without account',
 'View',
 'not-selected']

In [56]:
attribute_value_types

['Access Scope',
 'Access Type',
 'Action First-Party',
 'Action Third Party',
 'Audience Type',
 'Change Type',
 'Choice Scope',
 'Choice Type',
 'Collection Mode',
 'Do Not Track policy',
 'Does/Does Not',
 'Identifiability',
 'Notification Type',
 'Other Type',
 'Personal Information Type',
 'Purpose',
 'Retention Period',
 'Retention Purpose',
 'Security Measure',
 'Third Party Entity',
 'User Choice',
 'User Type']

In [57]:
attribute_value_values

['Additional service/feature',
 'Advertising',
 'Aggregated or anonymized',
 'Analytics/Research',
 'Basic service/feature',
 'Both',
 'Browser/device privacy controls',
 'Californians',
 'Children',
 'Citizens from other countries',
 'Collect from user on other websites',
 'Collect in mobile app',
 'Collect on first party website/app',
 'Collect on mobile website',
 'Collect on website',
 'Collection',
 'Computer information',
 'Contact',
 'Cookies and tracking elements',
 'Data access limitation',
 'Deactivate account',
 'Delete account (full)',
 'Delete account (partial)',
 'Demographic',
 'Does',
 'Does Not',
 'Dont use service/feature',
 'Edit information',
 'Europeans',
 'Explicit',
 'Export',
 'Financial',
 'First party collection',
 'First party use',
 'First-party privacy controls',
 'General notice in privacy policy',
 'General notice on website',
 'Generic',
 'Generic personal information',
 'Health',
 'Honored',
 'IP address and device IDs',
 'Identifiable',
 'Implicit',
 '

Okay, we have all of our attribute value types! Now, let's construct a dataframe of ALL of our data.

It should have columns: 

1. Paragraph2Vec representation of the text span
2. One hot representation of category

Unused data:
1. Segment index
2. Start index
3. End index
4. One hot representation of attribute
5. One hot representation of attribute type
6. One hot representation of attribute value


In [58]:
from nltk.tokenize import word_tokenize

In [59]:
model = gensim.models.Doc2Vec(size=50)

In [60]:
# flattened_policies = []
# for item in policies:
#     flattened_policies.extend(item)
# len(flattened_policies)

In [61]:
df_columns = ['text', 'category', 'category one hot', 'text vec']
df = pd.DataFrame([], columns=df_columns)
series = []
documents = []
cats = []
idx = 0
chosen_categories = ['First Party Collection/Use', 'Third Party Sharing/Collection']
remove_text = ['null', 'Not selected']
stemmer = nltk.stem.porter.PorterStemmer()
for datum in data:
    for idx in range(len(datum)):
        category = datum['Category Name'][idx]
        
        if chosen_categories is None or category not in chosen_categories:
            continue
        
        parsed = json.loads(datum['Attributes/Values'][idx])
        for value in attribute_value_types:
            if value in parsed.keys():
                attributes = parsed[value]
                if 'selectedText' in attributes:
                    text = attributes['selectedText']
                    
                    if text in remove_text:
                        continue
                    
                    text = text.lower()
                        
                    processed_text = word_tokenize(text)
                    processed_text = [stemmer.stem(word) for word in processed_text]
                    
                    text = ' '.join(processed_text)
                    
                    doc = gensim.models.doc2vec.TaggedDocument(processed_text, [idx])
                    documents.append(doc)
                    cats.append(cat_dict[category])
                    series.append(pd.Series([text, category, cat_dict[category], None], index=df_columns))
                    idx += 1
cats = np.array(cats)

In [62]:
documents[0]

TaggedDocument(words=['sci-news.com', 'may', 'collect', 'and', 'process'], tags=[1])

In [63]:
df = df.append(series, ignore_index=True)                    
df

,text,category,category one hot,text vec
0,sci-news.com may collect and process,First Party Collection/Use,"[0, 1]",None
1,nformat that you provid by fill in form on our...,First Party Collection/Use,"[0, 1]",None
2,includ,First Party Collection/Use,"[0, 1]",None
3,"other purpos , for exampl when you report a pr...",First Party Collection/Use,"[0, 1]",None
4,sci-news.com may collect and process,First Party Collection/Use,"[0, 1]",None
5,nformat that you provid by fill in form on our...,First Party Collection/Use,"[0, 1]",None
6,"name , e-mail and websit address",First Party Collection/Use,"[0, 1]",None
7,"other purpos , for exampl when you report a pr...",First Party Collection/Use,"[0, 1]",None
8,"includ , but not limit",First Party Collection/Use,"[0, 1]",None
9,weblog and other commun data,First Party Collection/Use,"[0, 1]",None


In [64]:
df.shape

(62335, 4)

In [65]:
model.build_vocab(documents)

In [66]:
model.train(documents, total_examples=len(documents), epochs=1)

453553

In [67]:
# setting model.random before infer_vector is required
# for determininistic behavior as described in 
# https://github.com/RaRe-Technologies/gensim/issues/447#issuecomment-138994654
model.random = np.random.RandomState(1234)
test_vec = model.infer_vector(word_tokenize('this is a test'))
print(test_vec.shape, '\n', test_vec)

(50,) 
 [-0.05887155 -0.00658131 -0.02952161 -0.04314176  0.05033931 -0.08453962
  0.04261466 -0.02260609 -0.03371405  0.0246499   0.02703367 -0.00660033
  0.0643023   0.00717241  0.08996434 -0.01332088 -0.03034603 -0.02817756
  0.01219391  0.01168366  0.01093591  0.01362334 -0.04622985 -0.03913774
 -0.02690259 -0.00309335 -0.01108695  0.00817398  0.04134864 -0.08806843
 -0.04072272  0.00027432 -0.03848925 -0.03097347  0.04408506 -0.01910358
  0.0443265   0.04377758 -0.09227458  0.05610784 -0.02899622  0.00650235
 -0.08142876 -0.04261572 -0.06436829  0.0331688  -0.01384362  0.01801005
 -0.01312876  0.01892142]


In [68]:
vecs = []
for row in df.itertuples():
    category_not_chosen = chosen_categories is None
    categorgy_chosen_and_matches = chosen_categories is not None and row.category in chosen_categories
    if categorgy_chosen_and_matches or category_not_chosen:
        model.random = np.random.RandomState(1234)
        vecs.append(np.array(model.infer_vector(word_tokenize(row.text))))
        
vecs = np.array(vecs)

In [69]:
vecs.shape

(62335, 50)

In [70]:
# df = pd.concat([df, pd.DataFrame(vecs)], axis=1)

In [71]:
# Keras version
nn_model = Sequential()
nn_model.reset_states()
nn_model.add(Dense(256, batch_input_shape=(None, 50, ), activation='relu'))
nn_model.add(Dropout(0.5))
nn_model.add(Dense(256, activation='relu'))
nn_model.add(Dropout(0.5))
#nn_model.add(Dense(10, activation='softmax'))
nn_model.add(Dense(2, activation='softmax'))

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
nn_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[metrics.mae, metrics.categorical_accuracy])

print(nn_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               13056     
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 514       
Total params: 79,362
Trainable params: 79,362
Non-trainable params: 0
_________________________________________________________________
None


In [72]:
cats.shape

(62335, 2)

In [73]:
df['category one hot'].shape

(62335,)

In [74]:
X_in = vecs
Y_in = cats
print(X_in.shape, Y_in.shape)

(62335, 50) (62335, 2)


In [75]:
tensorboard_callback = TensorBoard(log_dir='C:/tmp/pp_run-'+time.strftime("%Y-%m-%d-%H%M%S"))
nn_model.fit(X_in, Y_in, validation_split=0.25, batch_size=128, epochs=128, verbose=1, callbacks=[tensorboard_callback])

Train on 46751 samples, validate on 15584 samples
Epoch 1/128
46751/46751 [==============================] - 2s - loss: 0.6327 - mean_absolute_error: 0.4442 - categorical_accuracy: 0.6450 - val_loss: 0.6166 - val_mean_absolute_error: 0.4280 - val_categorical_accuracy: 0.6682
Epoch 2/128
46751/46751 [==============================] - 2s - loss: 0.6164 - mean_absolute_error: 0.4280 - categorical_accuracy: 0.6642 - val_loss: 0.6119 - val_mean_absolute_error: 0.4236 - val_categorical_accuracy: 0.6709
Epoch 3/128
46751/46751 [==============================] - 2s - loss: 0.6096 - mean_absolute_error: 0.4226 - categorical_accuracy: 0.6707 - val_loss: 0.6148 - val_mean_absolute_error: 0.4269 - val_categorical_accuracy: 0.6620
Epoch 4/128
46751/46751 [==============================] - 2s - loss: 0.6032 - mean_absolute_error: 0.4177 - categorical_accuracy: 0.6764 - val_loss: 0.6040 - val_mean_absolute_error: 0.4211 - val_categorical_accuracy: 0.6694
Epoch 5/128
46751/46751 [=====================

46751/46751 [==============================] - 2s - loss: 0.5415 - mean_absolute_error: 0.3680 - categorical_accuracy: 0.7204 - val_loss: 0.5649 - val_mean_absolute_error: 0.3822 - val_categorical_accuracy: 0.7008
Epoch 37/128
46751/46751 [==============================] - 2s - loss: 0.5423 - mean_absolute_error: 0.3676 - categorical_accuracy: 0.7202 - val_loss: 0.5614 - val_mean_absolute_error: 0.3821 - val_categorical_accuracy: 0.7066
Epoch 38/128
46751/46751 [==============================] - 2s - loss: 0.5391 - mean_absolute_error: 0.3654 - categorical_accuracy: 0.7230 - val_loss: 0.5620 - val_mean_absolute_error: 0.3786 - val_categorical_accuracy: 0.7026
Epoch 39/128
46751/46751 [==============================] - 3s - loss: 0.5407 - mean_absolute_error: 0.3663 - categorical_accuracy: 0.7218 - val_loss: 0.5632 - val_mean_absolute_error: 0.3797 - val_categorical_accuracy: 0.7017
Epoch 40/128
46751/46751 [==============================] - 2s - loss: 0.5399 - mean_absolute_error: 0.36

46751/46751 [==============================] - 2s - loss: 0.5195 - mean_absolute_error: 0.3499 - categorical_accuracy: 0.7353 - val_loss: 0.5595 - val_mean_absolute_error: 0.3814 - val_categorical_accuracy: 0.7037
Epoch 73/128
46751/46751 [==============================] - 2s - loss: 0.5174 - mean_absolute_error: 0.3489 - categorical_accuracy: 0.7361 - val_loss: 0.5635 - val_mean_absolute_error: 0.3799 - val_categorical_accuracy: 0.7019
Epoch 74/128
46751/46751 [==============================] - 2s - loss: 0.5170 - mean_absolute_error: 0.3482 - categorical_accuracy: 0.7375 - val_loss: 0.5641 - val_mean_absolute_error: 0.3780 - val_categorical_accuracy: 0.7015
Epoch 75/128
46751/46751 [==============================] - 2s - loss: 0.5185 - mean_absolute_error: 0.3492 - categorical_accuracy: 0.7360 - val_loss: 0.5662 - val_mean_absolute_error: 0.3789 - val_categorical_accuracy: 0.6997
Epoch 76/128
46751/46751 [==============================] - 2s - loss: 0.5185 - mean_absolute_error: 0.34

46751/46751 [==============================] - 3s - loss: 0.5019 - mean_absolute_error: 0.3368 - categorical_accuracy: 0.7456 - val_loss: 0.5614 - val_mean_absolute_error: 0.3743 - val_categorical_accuracy: 0.7010
Epoch 109/128
46751/46751 [==============================] - 2s - loss: 0.5024 - mean_absolute_error: 0.3365 - categorical_accuracy: 0.7491 - val_loss: 0.5644 - val_mean_absolute_error: 0.3739 - val_categorical_accuracy: 0.6987
Epoch 110/128
46751/46751 [==============================] - 3s - loss: 0.5025 - mean_absolute_error: 0.3370 - categorical_accuracy: 0.7472 - val_loss: 0.5671 - val_mean_absolute_error: 0.3801 - val_categorical_accuracy: 0.7014
Epoch 111/128
46751/46751 [==============================] - 3s - loss: 0.5022 - mean_absolute_error: 0.3369 - categorical_accuracy: 0.7459 - val_loss: 0.5642 - val_mean_absolute_error: 0.3765 - val_categorical_accuracy: 0.7019
Epoch 112/128
46751/46751 [==============================] - 3s - loss: 0.5024 - mean_absolute_error: 

In [84]:
test_text = 'We may disclose information'
#test_text = 'We do not share your profile with other third parties.'
#test_text = 'These tracking technologies may be deployed'
#test_text = 'we will not disclose your information'
#test_text = 'we use cookies or similar technologies'
#test_text = 'we collect your'
#test_text = 'share'

test_text = [stemmer.stem(word) for word in word_tokenize(test_text.lower())]
print(test_text)
model.random = np.random.RandomState(1234)
test_vec = model.infer_vector(test_text)
prediction_cat = nn_model.predict(np.array([test_vec])).round().astype(int)[0]
print(test_vec)
print(prediction_cat)
idx = 0
for one_hot_cat in one_hot_categories:
    if np.array_equal(one_hot_cat, prediction_cat):
        break
    idx += 1
print(idx)
print(chosen_categories[idx])

['we', 'may', 'disclos', 'inform']
[ -5.63437790e-02  -2.40227990e-02  -4.28044796e-02  -3.81142795e-02
   2.57552098e-02  -6.37648478e-02   4.27698381e-02  -7.14825615e-02
  -2.70575169e-03  -2.67965253e-03   7.32534006e-03   6.73667528e-03
  -1.08440919e-02   5.01380935e-02  -4.58504539e-03  -1.31346341e-02
   1.52812442e-02   2.48838421e-02  -4.09025513e-02  -2.68116016e-02
   7.13483095e-02   2.89816037e-02  -1.66882575e-02  -2.66089104e-02
  -4.68902476e-03   2.89001297e-02   2.17554811e-02   7.86258315e-05
   7.18803238e-03  -6.28447309e-02  -3.37776542e-02   5.12646660e-02
  -2.21613608e-03  -1.26664983e-02   2.29802411e-02  -1.52109368e-02
   2.10327413e-02   1.78241655e-02  -3.14462930e-02   2.02922188e-02
  -3.28240246e-02  -1.92398001e-02  -2.81555168e-02  -1.54720554e-02
  -1.03104091e-03  -9.48301703e-03  -2.53380686e-02   1.48004917e-02
   1.73599627e-02   4.25318256e-02]
[1 0]
1
Third Party Sharing/Collection
